In [57]:
import sys
sys.path.append('../_amigocloud')

In [58]:
import os
import pandas as pd

import win32com.client as win32

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from amigocloud import AmigoCloud

In [59]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [72]:
data_projects = ruta + '/_keys/amigo_cloud_projects.json'
data_projects_pd = pd.read_json(data_projects)

In [ ]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [61]:
# Ruta completa al archivo client_secrets.json
ruta_client_secrets = ruta + '\\_keys\\client_secrets.json'
gauth = GoogleAuth()
# Cargar configuración personalizada
gauth.LoadClientConfigFile(ruta_client_secrets)
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [68]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

In [69]:
def cargar_archivo_a_drive(file_name, file_path, directoryID):
    # Buscar archivos con el mismo nombre en el directorio
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(directoryID)}).GetList()
    for file in file_list:
        if file['title'] == file_name:
            # Si existe un archivo con el mismo nombre, borrarlo
            file.Trash()
    # crea el nuevo archivo
    file_drive = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": directoryID}]})
    file_drive['title'] = file_name
    file_drive.SetContentFile(file_path)
    file_drive.Upload() # Upload the file.
    #print('Created file %s with mimeType %s' % (file_drive['title'], file_drive['mimeType']))
    permission = file_drive.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    return file_drive['alternateLink']

In [66]:
def convert_to_pdf(file_path, output_path):
    # Inicializar la aplicación Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    word.Visible = False  # Corre Word en background
    # Abrir el documento
    doc = word.Documents.Open(file_path)
    doc.Activate()
    # Convertir a PDF
    word.ActiveDocument.ExportAsFixedFormat(output_path, win32.constants.wdExportFormatPDF)
    # Cerrar el documento y la aplicación
    doc.Close(False)
    word.Quit()

In [70]:
# Listar todos los archivos en la carpeta
archivos = os.listdir(ruta + '/informes/')

In [71]:
# Filtrar aquellos que no comienzan con un guion bajo
revisados = [archivo for archivo in archivos if not archivo.startswith('_')]
# Imprimir la lista filtrada
print(revisados)

['75_EMPC_01-04-2024_AGUILERA TARADELLES NELSON MARIANO_6.docx', '75_EMPC_15-04-2024_AGUILERA TARADELLES NELSON MARIANO_47.docx']


In [67]:

for file in revisados:
    id_reg = int(file.split('_')[-1].split('.')[0])
    sigla = file.split('_')[1]
    
    work_path = ruta + '/informes/'+ file  # Asegúrate de proporcionar la ruta completa
    pdf_path = ruta + '/pdf/'+ file.replace('.docx', '.pdf')  # Asegúrate de proporcionar la ruta completa
    convert_to_pdf(work_path, pdf_path)
    link_reporte = cargar_archivo_a_drive(file, pdf_path, '1Uy9TWXPzADH37WBwkjqqlhZgOaRwvQ6i')

    project = data_projects_pd[sigla]['proyecto']
    dataset = data_projects_pd[sigla]['dataset']
    column = data_projects_pd[sigla]['columna_link']

    query_sql = f"update dataset_{dataset} set {column} = '{link_reporte}' where id = {id_reg}"

    ejecutar_query_sql(project, query_sql, 'post')
    print('id', id_reg)
    

id 6
id 47


In [74]:

fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  if(file['title'] == "EVAL MANTENIMIENTO PREVENTIVO COSECHADORAS"):
      fileID = file['id']
fileID


'13_oMpP3Km28op_zkcmvMl0Z2dUF60sPc'